SPP (satellite precipitation products) - downscaling

period of 20 years from 2001 to 2020 included

# Setup

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate(auth_mode= 'notebook',  #force =True,
                #scopes='https://www.googleapis.com/auth/earthengine'
                )

ee.Initialize(project='mapping-applications')
import geemap

In [2]:
Map = geemap.Map()

In [3]:
gaul = ee.FeatureCollection('FAO/GAUL/2015/level2')

region = gaul.filter(ee.Filter.eq('ADM1_NAME', 'Sicilia' ));
geometry0 = region.geometry();
# this is the result from drawing a geom on the map and got from the 'user_roi' the coord info.
coordinates = [[[-9.755859, 30.524413], [-9.755859, 54.72462], [43.857422, 54.72462], [43.857422, 30.524413], [-9.755859, 30.524413]]]
geometry = ee.Geometry.Polygon(coordinates)

In [4]:
Map.addLayer(geometry0, {}, 'Sicilia')
#Map.addLayer(geometry,{}, 'area of downscaling')
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# downscaling method from original resolution to 1Km spatial resolution

in this study:
[Downscaling and merging multiple satellite precipitation products and gauge observations using random forest with the incorporation of spatial autocorrelation](https://www.sciencedirect.com/science/article/pii/S0022169424003135)

they used: IMERG, GSMaP, CHIRPS, MORPH, and PERSIANN for merging.

In [ ]:
# what i am using in here:

#  chirps (note that it incorporates some gauge stations observation world wide)
# "IDAHO_EPSCOR/TERRACLIMATE" ( note that: it incorporates the precipitation dataset from https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-monthly)
# persiann/noaa

In [ ]:
from datetime import datetime

# Set your parameters
time_start = '2001-01-01'
time_end = '2021-01-01'
years = range(2001, 2021)
months = range(1, 13)

# Function to aggregate monthly precipitation data
def create_monthly_precip_image(dataset, year, month):
    """
    Aggregates precipitation data for a specific dataset, year, and month.

    :param dataset: The precipitation dataset (ImageCollection)
    :param year: Year for aggregation
    :param month: Month for aggregation
    :return: Aggregated monthly precipitation Image
    """
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    monthly_precip = dataset.filterDate(start_date, end_date).sum()

    return monthly_precip \
        .toInt() \
        .set('system:time_start', start_date.millis()) \
        .set('year', year) \
        .set('month', month) \
        .set('system:index', start_date.format('YYYY-MM'))


# Function to generate monthly precipitation ImageCollection for any dataset
def generate_monthly_precip_collection(dataset):
    monthly_precip_images = []
    for year in years:
        for month in months:
            monthly_image = create_monthly_precip_image(dataset, year, month)
            monthly_precip_images.append(monthly_image)

    # Create an ImageCollection from the list of monthly images
    return ee.ImageCollection(monthly_precip_images)

## the original values
# NOAA PERSIANN CDR dataset
noaa_dataset = ee.ImageCollection("NOAA/PERSIANN-CDR") \
    .filter(ee.Filter.date(time_start, time_end)) \
    .select('precipitation')

# CHIRPS dataset
chirps_dataset = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD") \
    .filter(ee.Filter.date(time_start, time_end)) \
    .select('precipitation')

# IMERG dataset (NASA GPM)
imerg_dataset = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V07") \
    .filter(ee.Filter.date(time_start, time_end)) \
    .select('precipitation')

# TerraClimate dataset
terraClimate_dataset = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE") \
    .filter(ee.Filter.date(time_start, time_end)) \
    .select('pr') \
    .map(lambda image: image.rename('precipitation'))


# Generate monthly ImageCollections for each dataset
noaa_monthly_precip = generate_monthly_precip_collection(noaa_dataset)
chirps_monthly_precip = generate_monthly_precip_collection(chirps_dataset)
imerg_monthly_precip = generate_monthly_precip_collection(imerg_dataset)
terraclimate_monthly_precip = generate_monthly_precip_collection(terraClimate_dataset)

# Check number of images for each dataset
print(f'NOAA CDR monthly images: {noaa_monthly_precip.size().getInfo()}')
print(f'CHIRPS monthly images: {chirps_monthly_precip.size().getInfo()}')
print(f'IMERG monthly images: {imerg_monthly_precip.size().getInfo()}')
print(f'TerraClimate monthly images: {terraclimate_monthly_precip.size().getInfo()}')

NOAA CDR monthly images: 240
CHIRPS monthly images: 240
IMERG monthly images: 240
TerraClimate monthly images: 240


## Independent variables

In [ ]:
# independet variables preprocess
# Set your parameters
time_start = '2001-01-01'
time_end = '2021-01-01'
years = range(2001, 2021)
months = range(1, 13)

# Land Cover and Elevation (Static datasets)
lc = ee.ImageCollection("MODIS/061/MCD12Q1").mode().select('LC_Type1')  # Static for all time
dem = ee.Image("USGS/GTOPO30")  # Static for all time

# NDVI dataset (2000-2020)
ndvi = ee.ImageCollection("MODIS/061/MOD13A1") \
    .select('NDVI') \
    .filterDate(time_start, time_end)

# Temperature dataset (2000-2020)
temp = ee.ImageCollection("MODIS/061/MOD11A2") \
    .select('LST_Day_1km') \
    .filterDate(time_start, time_end)

# Precipitation dataset as a placeholder (for NOAA, CHIRPS, IMERG, TerraClimate, etc.)
# This can be replaced with any precipitation dataset
precip = ee.ImageCollection("NOAA/PERSIANN-CDR") \
    .filter(ee.Filter.date(time_start, time_end)) \
    .select('precipitation')

# Function to create monthly NDVI images
def create_ndvi_image(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    month_img = ndvi.filterDate(start_date, end_date).median()
    resample = month_img.resample('bilinear').reproject(crs=month_img.projection().crs(), scale=1000)

    return resample.multiply(0.0001).set('system:time_start', start_date.millis()).set('system:index', start_date.format('YYYY-MM'))

# Function to create monthly Temperature images
def create_temp_image(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    month_img = temp.filterDate(start_date, end_date).median()
    resample = month_img.resample('bilinear').reproject(crs=month_img.projection().crs(), scale=1000)

    return resample.multiply(0.02).set('system:time_start', start_date.millis()).set('system:index', start_date.format('YYYY-MM'))

# Generate NDVI, Temperature
ndvi_monthly_images = []
temp_monthly_images = []


for year in years:
    for month in months:
        # Create NDVI and Temperature images for each month
        ndvi_image = create_ndvi_image(year, month)
        temp_image = create_temp_image(year, month)

        # Append to the respective lists
        ndvi_monthly_images.append(ndvi_image)
        temp_monthly_images.append(temp_image)

# Convert lists into ImageCollections
ndvi_monthly = ee.ImageCollection(ndvi_monthly_images)
temp_monthly = ee.ImageCollection(temp_monthly_images)

# Check the size of the ImageCollections
print(f'NDVI monthly images: {ndvi_monthly.size().getInfo()}')
print(f'Temperature monthly images: {temp_monthly.size().getInfo()}')

NDVI monthly images: 240
Temperature monthly images: 240


## Downscaling model

In [ ]:
#downscaling model

# Define the function to train the model and downscale precipitation
def pr_model(img):
    band_names = img.bandNames()
    band_names_indep = band_names.remove('precipitation')

    # Increase sample size for better training
    training = img.stratifiedSample(
        numPoints=1000,
        classBand='precipitation',
        region=geometry,
        scale=1000,
        seed=42
    )
    # model tuning
    model = ee.Classifier.smileGradientTreeBoost(
        numberOfTrees=200,         # Increase to improve accuracy (with caution)
        shrinkage=0.005,           # Lower learning rate for better generalization (default = 0.005)
        samplingRate=0.6,          # randomness in sampling
        maxNodes=15,               # Limit the complexity of trees
        loss='LeastAbsoluteDeviation',
        seed=42                    # Consistent results at this specific seeding
    ).train(
        features=training,
        classProperty='precipitation',
        inputProperties=band_names
    ).setOutputMode('REGRESSION')

    pr1000 = img.select(band_names_indep).classify(model)

    return pr1000.rename('pr1000').addBands(
        img.select(['precipitation'], ['prOriginal'])
    ).copyProperties(img, img.propertyNames())

In [ ]:
# combine dependent and independet variables with the ML-Tree-based model

# I am going to use a two-steps approach with an ensamble tree-based method
 # step I : combine and downscale precip datasets (imerg, chirps, perisiann and terraclimate) one by one;
 # stepp II : merge the resulting 4 downscaled precip datasets with again tree-based ensemble model and get one merged result

In [ ]:
# Step I: Combine independent and dependent variables into single collections for each dataset

# 1. Combine for CHIRPS
collection_withChirps = (ndvi_monthly
                        .combine(temp_monthly)
                        .combine(chirps_monthly_precip)  # CHIRPS precipitation dataset
                        .select('precipitation')
                        .map(lambda img: img.addBands(dem).addBands(lc)
                            .copyProperties(img, img.propertyNames())))

# 2. Combine for IMERG
collection_withImerg = (ndvi_monthly
                        .combine(temp_monthly)
                        .combine(imerg_monthly_precip)  # IMERG precipitation dataset
                        .select('precipitation')
                        .map(lambda img: img.addBands(dem).addBands(lc)
                            .copyProperties(img, img.propertyNames())))

# 3. Combine for TerraClimate
collection_withTerraClimate = (ndvi_monthly
                        .combine(temp_monthly)
                        .combine(terraclimate_monthly_precip)  # TerraClimate precipitation dataset
                        .select('precipitation')
                        .map(lambda img: img.addBands(dem).addBands(lc)
                            .copyProperties(img, img.propertyNames())))
# 4. Combine for NOAA-PErsiann
collection_withNoaa = (ndvi_monthly
                        .combine(temp_monthly)
                        .combine(noaa_monthly_precip)
                        .select('precipitation')
                        .map(lambda img: img.addBands(dem).addBands(lc)
                            .copyProperties(img, img.propertyNames())))  # NOAA precipitation dataset

# Step II: Apply the model to each collection

# NOAA dataset already handled
noaa_modelled = collection_withNoaa.map(pr_model)

# CHIRPS dataset
chirps_modelled = collection_withChirps.map(pr_model)

# IMERG dataset
imerg_modelled = collection_withImerg.map(pr_model)

# TerraClimate dataset
terraClimate_modelled = collection_withTerraClimate.map(pr_model)

# Step III: Simplify the region geometry for visualization
simplified_region = region.map(lambda feature: feature.simplify(1000))
roi = simplified_region.geometry()

# Step IV: Calculate the mean of 'pr1000' for each modelled dataset

# NOAA
pr_noaa_mean = noaa_modelled.select('pr1000').mean()

# CHIRPS
pr_chirps_mean = chirps_modelled.select('pr1000').mean()

# IMERG
pr_imerg_mean = imerg_modelled.select('pr1000').mean()

# TerraClimate
pr_terraClimate_mean = terraClimate_modelled.select('pr1000').mean()

# Step V: Define the sharpening function for each dataset

# Sharpen function for NOAA
def noaa_sharpen_image(img):
    corrected = img.select('pr1000').add(img.select('prOriginal').subtract(pr_noaa_mean))
    corrected_positive = corrected.where(corrected.lt(0), 0)
    return corrected_positive.copyProperties(img, img.propertyNames())

# Sharpen function for CHIRPS
def chirps_sharpen_image(img):
    corrected = img.select('pr1000').add(img.select('prOriginal').subtract(pr_chirps_mean))
    corrected_positive = corrected.where(corrected.lt(0), 0)
    return corrected_positive.copyProperties(img, img.propertyNames())

# Sharpen function for IMERG
def imerg_sharpen_image(img):
    corrected = img.select('pr1000').add(img.select('prOriginal').subtract(pr_imerg_mean))
    corrected_positive = corrected.where(corrected.lt(0), 0)
    return corrected_positive.copyProperties(img, img.propertyNames())

# Sharpen function for TerraClimate
def terraClimate_sharpen_image(img):
    corrected = img.select('pr1000').add(img.select('prOriginal').subtract(pr_terraClimate_mean))
    corrected_positive = corrected.where(corrected.lt(0), 0)
    return corrected_positive.copyProperties(img, img.propertyNames())

# Step VI: Apply the sharpening function to each modelled dataset

noaa_1km_cor = noaa_modelled.map(noaa_sharpen_image)
chirps_1km_cor = chirps_modelled.map(chirps_sharpen_image)
imerg_1km_cor = imerg_modelled.map(imerg_sharpen_image)
terraClimate_1km_cor = terraClimate_modelled.map(terraClimate_sharpen_image)

In [ ]:
#print(noaa_modelled.getInfo())
#print(chirps_modelled.getInfo())
#print(imerg_modelled.getInfo())
#print(terraClimate_modelled.getInfo())

In [ ]:
#terraClimate_1km_cor

In [ ]:
visPrecip = {'color': ['#a50026', ' #a70226', ' #a90426', ' #ab0626', ' #ad0826', ' #af0926', ' #b10b26', ' #b30d26', ' #b50f26', ' #b71126', ' #b91326', ' #bb1526', ' #bd1726', ' #be1827', ' #c01a27', ' #c21c27', ' #c41e27', ' #c62027', ' #c82227', ' #ca2427', ' #cc2627', ' #ce2827', ' #d02927', ' #d22b27', ' #d42d27', ' #d62f27', ' #d83128', ' #d93429', ' #da362a', ' #db382b', ' #dc3b2c', ' #dd3d2d', ' #de402e', ' #e0422f', ' #e14430', ' #e24731', ' #e34933', ' #e44c34', ' #e54e35', ' #e65036', ' #e75337', ' #e95538', ' #ea5739', ' #eb5a3a', ' #ec5c3b', ' #ed5f3c', ' #ee613e', ' #ef633f', ' #f16640', ' #f26841', ' #f36b42', ' #f46d43', ' #f47044', ' #f57245', ' #f57547', ' #f57748', ' #f67a49', ' #f67c4a', ' #f67f4b', ' #f7814c', ' #f7844e', ' #f8864f', ' #f88950', ' #f88c51', ' #f98e52', ' #f99153', ' #f99355', ' #fa9656', ' #fa9857', ' #fa9b58', ' #fb9d59', ' #fba05b', ' #fba35c', ' #fca55d', ' #fca85e', ' #fcaa5f', ' #fdad60', ' #fdaf62', ' #fdb164', ' #fdb366', ' #fdb567', ' #fdb769', ' #fdb96b', ' #fdbb6d', ' #fdbd6f', ' #fdbf71', ' #fdc173', ' #fdc374', ' #fdc576', ' #fdc778', ' #fec87a', ' #feca7c', ' #fecc7e', ' #fece7f', ' #fed081', ' #fed283', ' #fed485', ' #fed687', ' #fed889', ' #feda8a', ' #fedc8c', ' #fede8e', ' #fee090', ' #fee192', ' #fee294', ' #fee496', ' #fee597', ' #fee699', ' #fee79b', ' #fee99d', ' #feea9f', ' #feeba1', ' #feeca2', ' #feeda4', ' #feefa6', ' #fff0a8', ' #fff1aa', ' #fff2ac', ' #fff3ad', ' #fff5af', ' #fff6b1', ' #fff7b3', ' #fff8b5', ' #fffab7', ' #fffbb9', ' #fffcba', ' #fffdbc', ' #fffebe', ' #feffc0', ' #fdfec2', ' #fcfec5', ' #fbfdc7', ' #fafdc9', ' #f8fccb', ' #f7fcce', ' #f6fbd0', ' #f5fbd2', ' #f3fbd4', ' #f2fad6', ' #f1fad9', ' #f0f9db', ' #eff9dd', ' #edf8df', ' #ecf8e2', ' #ebf7e4', ' #eaf7e6', ' #e9f6e8', ' #e7f6eb', ' #e6f5ed', ' #e5f5ef', ' #e4f4f1', ' #e2f4f4', ' #e1f3f6', ' #e0f3f8', ' #def2f7', ' #dcf1f7', ' #daf0f6', ' #d8eff6', ' #d6eef5', ' #d4edf4', ' #d1ecf4', ' #cfebf3', ' #cdeaf3', ' #cbe9f2', ' #c9e8f2', ' #c7e7f1', ' #c5e6f0', ' #c3e5f0', ' #c1e4ef', ' #bfe3ef', ' #bde2ee', ' #bbe1ed', ' #b9e0ed', ' #b6dfec', ' #b4deec', ' #b2ddeb', ' #b0dcea', ' #aedbea', ' #acdae9', ' #aad8e9', ' #a8d6e8', ' #a6d5e7', ' #a3d3e6', ' #a1d1e5', ' #9fd0e4', ' #9dcee3', ' #9bcce2', ' #99cae1', ' #97c9e0', ' #94c7df', ' #92c5de', ' #90c3dd', ' #8ec2dc', ' #8cc0db', ' #8abeda', ' #87bdd9', ' #85bbd9', ' #83b9d8', ' #81b7d7', ' #7fb6d6', ' #7db4d5', ' #7ab2d4', ' #78b0d3', ' #76afd2', ' #74add1', ' #72abd0', ' #70a9cf', ' #6ea6ce', ' #6da4cc', ' #6ba2cb', ' #69a0ca', ' #679ec9', ' #659bc8', ' #6399c7', ' #6297c6', ' #6095c4', ' #5e93c3', ' #5c90c2', ' #5a8ec1', ' #588cc0', ' #578abf', ' #5588be', ' #5385bd', ' #5183bb', ' #4f81ba', ' #4d7fb9', ' #4b7db8', ' #4a7ab7', ' #4878b6', ' #4676b5', ' #4574b3', ' #4471b2', ' #436fb1', ' #426cb0', ' #416aaf', ' #4167ad', ' #4065ac', ' #3f62ab', ' #3e60aa', ' #3e5ea8', ' #3d5ba7', ' #3c59a6', ' #3b56a5', ' #3a54a4', ' #3a51a2', ' #394fa1', ' #384ca0', ' #374a9f', ' #36479e', ' #36459c', ' #35429b', ' #34409a', ' #333d99', ' #333b97', ' #323896', ' #313695'], 'min': 61.73221853877888, 'max': 121.5413563055337, 'opacity': 1.0}

Map.addLayer(noaa_1km_cor.toBands().clip(region), visPrecip, 'noaa_20y- 1km', 0)

Map.addLayer(chirps_1km_cor.toBands().clip(region), visPrecip, 'chirps_20y - 1km', 0)

Map.addLayer(terraClimate_1km_cor.toBands().clip(region), visPrecip, 'terraClimate_20y - 1km', 0)
#Map